In [1]:
import torch
from transformers import BertModel, BertTokenizer, WordpieceTokenizer
import os

/home/btech/rsrivatsa/miniconda3/envs/pytorch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/btech/rsrivatsa/miniconda3/envs/pytorch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/btech/rsrivatsa/miniconda3/envs/pytorch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/btech/r

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"
device='cuda:2'
device = torch.device(device if torch.cuda.is_available() else "cpu")

In [3]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

In [9]:
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)

In [5]:
model = BertModel.from_pretrained(pretrained_weights)
model=model.to(device)

In [6]:
# input_ids = torch.tensor([tokenizer.encode("Let's <user> see all hidden-states and attentions on this text", add_special_tokens=True, max_length=512)])

In [7]:
# a =  model(input_ids)

In [4]:
import pandas as pd
import numpy as np
import os
import pickle
import numpy as np
import re
import itertools
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [5]:
train_path_DAC = './Data/SWBD.csv'
dataframe=pd.read_csv(train_path_DAC)
X_train_swbd=dataframe['TEXT'].fillna('<UNK>').values
Y_train_swbd=dataframe['NEW_TAX'].fillna('<UNK>').values

test_path_DAC='./Data/SWBD_valid.csv'
dataframe=pd.read_csv(test_path_DAC)
X_test_swbd=dataframe['TEXT'].fillna('<UNK>').values
Y_test_swbd=dataframe['NEW_TAX'].fillna('<UNK>').values

In [6]:
train_path_tw = './Data/TWEET_DATA1.csv'
dataframe=pd.read_csv(train_path_tw)
X_train_tw=dataframe['TEXT'].fillna('<UNK>').values
Y_train_tw=dataframe['Label'].fillna('<UNK>').values

In [7]:
X_train_swbd = [s.strip() for s in X_train_swbd]
X_test_swbd = [s.strip() for s in X_test_swbd]
X_train_tw = [s.strip() for s in X_train_tw]

In [10]:
X_train_swbd_inp_ids = [  tokenizer.encode(sent, add_special_tokens=True, max_length=128) for sent in X_train_swbd  ]

In [11]:
X_test_swbd_inp_ids = [  tokenizer.encode(sent, add_special_tokens=True, max_length=128) for sent in X_test_swbd  ]
X_train_tw_inp_ids = [  tokenizer.encode(sent, add_special_tokens=True, max_length=128) for sent in X_train_tw  ]

In [12]:
maxLen = 128

In [13]:
X_train_swbd_inp_ids_padded = pad_sequences(X_train_swbd_inp_ids, maxlen=maxLen, dtype="long", truncating="post", padding="post")
X_test_swbd_inp_ids_padded = pad_sequences(X_test_swbd_inp_ids, maxlen=maxLen, dtype="long", truncating="post", padding="post")
X_train_tw_inp_ids_padded = pad_sequences(X_train_tw_inp_ids, maxlen=maxLen, dtype="long", truncating="post", padding="post")

In [14]:
def retAttnMask(input_ids):
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)
    return attention_masks

In [15]:
X_train_swbd_inp_ids_attMask = retAttnMask(X_train_swbd_inp_ids_padded)

X_test_swbd_inp_ids_attMask = retAttnMask(X_test_swbd_inp_ids_padded)

X_train_tw_inp_ids_attMask = retAttnMask(X_train_tw_inp_ids_padded)

In [18]:
def getData(X, M):
    x_t = torch.tensor(X)
    m_t = torch.tensor(M)
    mydata = TensorDataset(x_t, m_t)
    data_sampler = SequentialSampler(mydata)
    data_loader = DataLoader(mydata, sampler=data_sampler, batch_size=32)
    return data_loader

swbd_tr_dl = getData(X_train_swbd_inp_ids_padded, X_train_swbd_inp_ids_attMask)

swbd_te_dl = getData(X_test_swbd_inp_ids_padded, X_test_swbd_inp_ids_attMask)

tw_tr_dl = getData(X_train_tw_inp_ids_padded, X_train_tw_inp_ids_attMask)

model.eval()

def getEmbeddings(dl):
    sqbd_tr_emb = []
    cnt = 0
    for batch in dl:
        cnt+=1
        if cnt%5 == 0:
            print(cnt, end=" ")
        
        with torch.no_grad():
            b_input_ids, b_input_mask = batch
            b_input_ids, b_input_mask = b_input_ids.to(device), b_input_mask.to(device)
            myout, _ = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            myout = myout.to(torch.device("cpu"))
            sqbd_tr_emb.append(myout.numpy())
    return sqbd_tr_emb

X_train_tw_inp_ids_padded.shape

b = getEmbeddings(swbd_te_dl)

with open('Data/swbd_tr/swbd_tr_emb.pkl','wb') as fd:
    pickle.dump([X_train_swbd_inp_ids, b],fd)

print('hi')

5 10 15 20 25 30 35 40 45 50 55 60 65 70 75 80 85 90 95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 

KeyboardInterrupt: 

In [16]:
def get_split_inds(path):
    with open(path,'r') as f:
        a = f.readlines()
    a = a[1:]
    cnt = 0
    sp_ind = [0]
    for lin in a:
        if lin == '\n':
            sp_ind.append(cnt)
        else:
            cnt+=1
    return sp_ind


def split_data_dialog(values,unpaded_values,labels,sp_inds):
    li = []
    yli = []
    uli = []
    dial_len = []
    sentLen = []
    for i in range(len(sp_inds)-1):
        li.append(values[sp_inds[i]:sp_inds[i+1]])
        yli.append(labels[sp_inds[i]:sp_inds[i+1]])
        uli.append(unpaded_values[sp_inds[i]:sp_inds[i+1]])
        
        dial_len.append(len(li[-1]))
        st = []
        for j in unpaded_values[sp_inds[i]:sp_inds[i+1]]:
            st.append(len(j))
        sentLen.append(np.asarray(st,dtype = np.int32))
            
    return li,yli,uli,dial_len,sentLen

In [17]:
def split_data_dialog_tw(values,unpaded_values,labels,sp_inds, exf):
    li = []
    yli = []
    uli = []
    dial_len = []
    sentLen = []
    exfList = []
    for i in range(len(sp_inds)-1):
        li.append(values[sp_inds[i]:sp_inds[i+1]])
        exfList.append(exf[sp_inds[i]:sp_inds[i+1]])
        yli.append(labels[sp_inds[i]:sp_inds[i+1]])
        uli.append(unpaded_values[sp_inds[i]:sp_inds[i+1]])
        
        dial_len.append(len(li[-1]))
        st = []
        for j in unpaded_values[sp_inds[i]:sp_inds[i+1]]:
            st.append(len(j))
        sentLen.append(np.asarray(st,dtype = np.int32))
            
    return li,yli,uli,dial_len,sentLen,exfList

In [18]:
def create_label(label,lbl_Dict = None):
    
    if lbl_Dict is None:
        lbl_dict={}
        index=0
        for dial_lbls in label:
            if dial_lbls not in lbl_dict:
                lbl_dict[dial_lbls]=index
                index=index+1
    else:
        lbl_dict = lbl_Dict
    print(lbl_dict)
    Y=[]
    for i in label:
        xxx=np.zeros(int(len(lbl_dict)))
        j=lbl_dict.get(i)
        xxx[j]=1
        Y.append(xxx)
    return Y,lbl_dict

In [19]:
Y_train_swbd,ldic_swbd = create_label(Y_train_swbd,None)
Y_test_swbd,_ = create_label(Y_test_swbd,ldic_swbd)
Y_train_tw,ldic_tw = create_label(Y_train_tw,None) 

{'CAT': 0, 'QUESTION': 1, 'INFO': 2, 'COMMAND': 3, 'General_Greeting': 4}
{'CAT': 0, 'QUESTION': 1, 'INFO': 2, 'COMMAND': 3, 'General_Greeting': 4}
{'STM': 0, 'EXP': 1, 'QUE': 2, 'OTH': 3, 'SUG': 4, 'REQ': 5, 'THT': 6}


In [20]:
ind_tr_swbd = get_split_inds(train_path_DAC)
ind_te_swbd = get_split_inds(test_path_DAC)

In [24]:
num = 7500//32
ind_tr_tw = [ (i+1)*32 for i in range(num)]
ind_tr_tw = [0] +  ind_tr_tw
ind_tr_tw.append(7500)

In [21]:
X_tr_List_swbd= split_data_dialog(np.arange(len(X_train_swbd_inp_ids_padded)),X_train_swbd_inp_ids,Y_train_swbd,ind_tr_swbd)

In [22]:
X_te_List_swbd= split_data_dialog(np.arange(len(X_test_swbd_inp_ids_padded)),X_test_swbd_inp_ids,Y_test_swbd,ind_te_swbd)

In [45]:
x_tr_List_tw = split_data_dialog_tw(np.arange(len(X_train_tw_inp_ids_padded)),X_train_tw_inp_ids,Y_train_tw,ind_tr_tw, )

In [ ]:
len(x_tr_List_tw[-1])

In [47]:
# X_te_List = split_data_dialog(c,X_test_swbd_unpadded,Y_test_swbd,ind_te_swbd)


In [48]:
# X_te_List = split_data_dialog(c,X_test_swbd_inp_ids,Y_test_swbd,ind_te_swbd)

In [27]:
def save_each_dial(X_list, savepath):
    li  = X_list[0]
    lab = X_list[1]
    
    for i,dial in enumerate(li):
        np.save(savepath +'/X_'+str(i) +'.npy', np.asarray(dial, dtype = np.float32))
        np.save(savepath +'/Y_'+str(i) +'.npy', np.asarray(lab[i], dtype = np.int32))
        

In [28]:
with open('Data/final_data.pkl','rb') as g:
    extafeats = pickle.load(g)

In [29]:
extrafeats = [np.asarray(b.split(" "),dtype = np.float32) for b in extafeats]

In [30]:
extrafeats = np.asarray(extrafeats,dtype = np.float32)

In [31]:
extrafeats.shape

(7500, 4110)

In [46]:
x_tr_List_tw = split_data_dialog_tw(np.arange(len(X_train_tw_inp_ids_padded)),X_train_tw_inp_ids,Y_train_tw,ind_tr_tw, extrafeats)

In [47]:
x_tr_List_tw = list(x_tr_List_tw)

In [50]:
len(x_tr_List_tw)

7

In [49]:
x_tr_List_tw.append(range(len(x_tr_List_tw[0])) )

In [52]:
for i in x_tr_List_tw:
    print(len(i))

235
235
235
235
235
235
235


In [54]:
from sklearn.model_selection import train_test_split

In [57]:
tw_tr_val = train_test_split(*x_tr_List_tw,test_size=0.2,random_state=42)

In [59]:
type(tw_tr_val)

list

In [60]:
to_save_tw_train = [
    tw_tr_val[0],
    tw_tr_val[2],
    tw_tr_val[8],
    tw_tr_val[6],
    ldic_tw,
    tw_tr_val[10],
    tw_tr_val[12],
]

In [61]:
to_save_tw_test = [
    tw_tr_val[1],
    tw_tr_val[3],
    tw_tr_val[9],
    tw_tr_val[7],
    ldic_tw,
    tw_tr_val[11],
    tw_tr_val[13],
]

In [64]:
with open('Data/twit_validset_bert.pkl','wb') as f:
    pickle.dump(to_save_tw_test,f)
with open('Data/twit_trainset_bert.pkl','wb') as f:
    pickle.dump(to_save_tw_train,f)

In [70]:
# Data/swbd_trainset.pkl
toSave_swbd_train = [
    X_tr_List_swbd[0],
    X_tr_List_swbd[1],
    X_tr_List_swbd[4],
    X_tr_List_swbd[3],
    ldic_swbd,
    range(len(X_tr_List_swbd[0])) 
]

toSave_swbd_test = [
    X_te_List_swbd[0],
    X_te_List_swbd[1],
    X_te_List_swbd[4],
    X_te_List_swbd[3],
    ldic_swbd,
    range(len(X_te_List_swbd[0])) 
]

112

In [80]:
with open('Data/swbd_validset_bert.pkl','wb') as f:
    pickle.dump(toSave_swbd_test,f)

In [81]:
with open('Data/swbd_trainset_bert.pkl','wb') as f:
    pickle.dump(toSave_swbd_train,f)